In [1]:
# from entmoot import Enting, ProblemConfig, PyomoOptimizer
# import numpy as np
# import random

# # This is the function to be minimized. Usually some complicated like a simulation.
# def my_func(x: float) -> float:
#     return x**2 + 1 + random.uniform(-0.2, 0.2)

# # Define a one-dimensional minimization problem with one real variable bounded by -2 and 3
# problem_config = ProblemConfig()
# problem_config.add_feature("real", (-2, 3))
# problem_config.add_min_objective()

# # Create training data for the tree model
# X_train = np.reshape(np.linspace(-2, 3, 10), (-1, 1))
# y_train = np.reshape([my_func(x) for x in X_train], (-1, 1))

# # Define Bayesian optimization parameters using an l1-distance based uncertainty measure and
# # penalizing the distance from well-known areas, i.e. exploitation (instead of exploration)
# params_bo = {"unc_params": {"dist_metric": "l1", "acq_sense": "penalty"}}

# # Define an Enting object which holds information about the problem as well as the parameters...
# enting = Enting(problem_config, params=params_bo)
# # ... and train the underlying tree model.
# enting.fit(X_train, y_train)

# # Create an PyomoOptimizer object that solves the optimization problem using the solver "GLPK"
# params_pyo = {"solver_name": "glpk"}
# opt_pyo = PyomoOptimizer(problem_config, params=params_pyo)
# res = opt_pyo.solve(enting)

# # Inspect the result. The optimal point should be close to zero.
# print(f"Optimal point: {res.opt_point[0]}")

### Translation into our framework

In [2]:
###########
# Imports #
###########

# importing libraries
import matplotlib.pyplot as plt
import copy
# importing algorithms
from BFGS_multistart import*
from Stochastic_local_search import*
from Cuadratic_opt_v2 import *
from Scipy_opt_algs import*
from BO_NpScpy import*
# importing test functions
from test_function import*
from utils import *
from COBYQA import *
from ScikitQuant_opt_algs import *
from pySOT_opt_algs import *
from CUATRO import *
from ENTMOOT import *


##########################
# Algorithms & Functions #
##########################

home_dir  = 'images/trajectory_plots_1D_&_tables'

algorithms_test = [
    # LS_QM_v2,
    # opt_SnobFit,
    # opt_SRBF,
    # opt_DYCORS,
    # opt_SOP,
    # opt_COBYLA,
    # COBYQA,
    # opt_CUATRO,
# # # Feel free to add these commented-out algorithms as a further test,
# # # however, do note they are more time-consuming to run
# BO_np_scipy,
opt_ENTMOOT,
]

functions_test  = [  #!!! Multimodal before Unimodal!!!
    'Levy_f',
    'Ackley_f',
    'Rosenbrock_f',
    'Antonio_f',
    ]

multim  = ['Levy_f', 'Ackley_f']
unim    = ['Rosenbrock_f', 'Antonio_f']

###########################
# Optimization parameters #
###########################

N_x_l        = [
    2,
    5,
    7,
    ]               # Number of input dimensions

start_       = [5, 10, 15]              # starting points on the trajectory of the algorithm on the test function

reps         = 5                        # We run each algorithm 5 times on each objective function. The number of evaluations per run is given by f_eval_l
f_eval_l     = [
    20,
    50,
    100
    ]  # These are the number of function evaluations (length of trajectory) foreach run in reps. The number depends on the input dimension N_x_l

In [3]:
x_shift = np.array([[-0.65436022], [ 2.09921755]])
n_x = 2
bounds = np.array([[-7., 7.], [-7., 7.]])

In [4]:
t_ = Test_function(
    x_shift=x_shift,
    func_type='Rosenbrock_f', 
    n_x=n_x, 
    track_x=False, 
    bounds=bounds
    )

### our version

In [6]:
from entmoot import Enting, ProblemConfig, GurobiOptimizer
import numpy as np

# define problem
problem_config = ProblemConfig(rnd_seed=1234)

# specify dimensions and ranges
for d_ in range(n_x):

    bounds_tuple = tuple(bounds[d_])
    problem_config.add_feature("real", bounds_tuple)

# specify goal
# CHECK HERE IF THIS IS THE CORRECT DIRECTION
problem_config.add_min_objective()

# specify initial no. of samples
N_init_samples = 15

# generate initial samples
train_y,train_x = Random_searchENT(t_.fun_test, n_p=n_x, bounds_rs=bounds, iter_rs=N_init_samples)


In [7]:
# get training-dataset in the form required for the fit method of enting
# x is a list of tuples, which represent one datapoint each. A 2D-problem has 2 entries per tuple
# y is a numpy array of shape (n_points, 1)

train_y_cust = train_y.reshape((len(train_y),1))
train_x_cust = list(zip(*train_x))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-22
Set parameter MIPGap to value 0
129.6753709787268
Set parameter MIPGap to value 0
79.76561030015527
Set parameter MIPGap to value 0
301.24340957872136
Set parameter MIPGap to value 0
170.96358656301746
Set parameter MIPGap to value 0
108.8238459128776
Set parameter MIPGap to value 0
103.7483072294721
Set parameter MIPGap to value 0
55.22615037771871
Set parameter MIPGap to value 0
83.602789391827
Set parameter MIPGap to value 0
101.48685307528652
Set parameter MIPGap to value 0
69.5735717192311
Set parameter MIPGap to value 0
108.80132609915931
Set parameter MIPGap to value 0
79.34768764142781
Set parameter MIPGap to value 0
66.7338839934547
Set parameter MIPGap to value 0
72.78123507857221
Set parameter MIPGap to value 0
61.224251104273954
Set parameter MIPGap to value 0
57.91055901340951
Set parameter MIPGap to value 0
71.50042577253711
Set parameter MIPGap to value 0
55.485753144164796
Set par

### their version

In [9]:
# import numpy as np


# def blackbox_ground_truth(x: np.ndarray, noise_std: float = 0.02) -> np.ndarray:
#     """Get the ground truth response for each observation in a dataset

#     The input array is N_observations x N_input_features and the output
#     of this function is a column vector with N_observations elements

#     Note for Nathan: I think this problem I invented is not a good one
#     to demonstrate GBT complexity. I'm sure you have a better idea.
#     """

#     def evaluate_blackbox_single_obs(
#         xin: np.ndarray
#     ) -> np.ndarray:
#         return (
#             xin[0]
#             + 0.25 * xin[1] * xin[2]
#             - 0.33 * xin[0] * xin[1] * xin[2]
#             + 0.2 * xin[3]
#             - 0.5 * xin[1] * xin[3]
#             + np.sin(4.0 * xin[1] * np.pi)
#             + np.cos(4.0 * xin[0] * xin[3] * np.pi)
#             + (xin[1] - 0.6) ** 2
#             + (xin[3] - 0.2) ** 2
#             + (xin[2] - 0.85)**2
#             + np.random.normal(loc=0, scale=noise_std)
#             + xin[0]*np.exp(xin[1] * xin[2])
#         )

#     return np.array([evaluate_blackbox_single_obs(xobs) for xobs in x]).reshape(-1, 1)

In [10]:
# from entmoot import Enting, ProblemConfig, GurobiOptimizer
# import numpy as np

# # define problem
# problem_config = ProblemConfig(rnd_seed=1234)
# problem_config.add_feature("real", (-4, 4))
# problem_config.add_feature("real", (-4, 4))
# problem_config.add_feature("real", (-4, 4))
# problem_config.add_feature("real", (-4, 4))
# problem_config.add_min_objective()

# # sample data for training. This is the intial dataset from before the
# # iterative optimization starts. Let's make it more interesting by avoiding
# # any really good points (obj < 0) during the initial sampling. Do this in a
# # quick and dirty way assuming that we will randomly generate enough samples

# N_init_samples = 15
# train_x = problem_config.get_rnd_sample_list(num_samples=N_init_samples * 100)
# # train_y,train_x = Random_searchENT(blackbox_ground_truth, n_p=n_x, bounds_rs=bounds, iter_rs=N_init_samples)

# train_y = blackbox_ground_truth(train_x) # supposed to be a numpy array of shape (N_init_samples,1)
# keep = train_y > 0
# train_x = [train_x[idx] for idx, k in enumerate(keep) if k][:N_init_samples]
# train_y = train_y[keep][:N_init_samples].reshape(-1, 1)



In [11]:
# from itertools import chain
# import gurobipy as gp

# N_iterations = 50
# params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration", "beta": 1.5}}
# enting = Enting(problem_config, params=params)
# params_gurobi = {"MIPGap": 0, "OutputFlag": 0}
# opt_gur = GurobiOptimizer(problem_config, params=params_gurobi)

# # Remember the proposals and outcomes in these variables
# opt_trajectory_inputs = []
# opt_trajectory_outputs = np.empty((N_iterations, 1))

# # train_x and train_y are from before


# for idx in range(N_iterations):
#     # Put together the initial dataset and any optimization iterations
#     # we have done so far
#     x = [_ for _ in chain(train_x, opt_trajectory_inputs)]
#     y = np.concatenate(
#         (train_y, opt_trajectory_outputs[:idx, :]), axis=0
#     )


#     enting.fit(x, y) 
#     res_gur = opt_gur.solve(enting)
#     opt_trajectory_inputs.append(tuple(xopt for xopt in res_gur.opt_point)) # opt_point is a list of length n_x (dimensions of x)
#     print(blackbox_ground_truth(opt_trajectory_inputs)[-1,0])
#     opt_trajectory_outputs[idx, 0] = blackbox_ground_truth(opt_trajectory_inputs)[-1,0]

### my version

In [12]:
# # Very dense sampling to find the optimum empirically for later comparison
# N_dense_samples = 100000
# dense_sample = problem_config.get_rnd_sample_list(num_samples=N_dense_samples)
# dense_sample_outcomes = blackbox_ground_truth(dense_sample, noise_std=0)

# opt_objval = np.min(dense_sample_outcomes)
# print(
#     f"Function has a minimum value of approx {np.round(opt_objval ,decimals=3)}",
#     f" at position \n{[np.round(x, decimals=3) for x in dense_sample[np.argmin(dense_sample_outcomes)]]}",)